In [1]:
import sqlite3 as sql
import pandas as pd
import codecs, re

In [2]:

db = sql.connect('database.sqlite')
data = pd.read_sql('SELECT parent_id, body, id FROM May2015\
                    WHERE LENGTH(body) > 10 AND LENGTH(body) < 150', db, chunksize=100000)


In [8]:
REMOVE = re.compile(r'[\*\r\n]*')

In [34]:
def getParent(parent_id):
    return pd.read_sql("SELECT parent_id, body, id FROM May2015\
                        WHERE id='" + comment.parent_id[3:] + "'", db)
    
def validComment(comment):
    
    # ignore deleted comments
    if '[deleted]' in comment:
        return False
    
    # allow only ascii characters
    try:
        comment.decode('ascii')
    except:
        return False
    
    # no emoticons allowed!
#     if re.match('\[\]\(/[a-z]+\)', emoticon) is None:
    if '[]' in comment[:2]:
        return False
    
    return True
        
    
minconvlen = 8
numchunks = 0
filenum = 0


with open('conv/redditComments.txt', 'w+') as f:
    # for each chunk of data
    for chunk in data:

        # for each comment in the chunk
        for idx, comment in chunk.iterrows():
            parent = getParent(comment.parent_id)

            if not validComment(comment.body):
                continue

            # trace parents
            conv = [comment.body]
            while not parent.empty:

                if not validComment(parent.body[0]):
                    break

                conv.append(parent.body[0])
                parent = getParent(parent.parent_id[0])

            # write long conversations to file
            if len(conv) > minconvlen:
                for comm in reversed(conv):
                    f.write(re.sub(REMOVE, ' ', comm))
                    f.write('\n')

                filenum += 1

        numchunks +=1
        print "Chunks processed:\t" + str(numchunks), "Files written:\t" + str(filenum)

Chunks processed:	1 Files written:	406
Chunks processed:	2 Files written:	1442
Chunks processed:	3 Files written:	2812
Chunks processed:	4 Files written:	4060
Chunks processed:	5 Files written:	5757
Chunks processed:	6 Files written:	7416
Chunks processed:	7 Files written:	9229
Chunks processed:	8 Files written:	11544
Chunks processed:	9 Files written:	14197
Chunks processed:	10 Files written:	17187
Chunks processed:	11 Files written:	20368
Chunks processed:	12 Files written:	22808
Chunks processed:	13 Files written:	25150
Chunks processed:	14 Files written:	27655
Chunks processed:	15 Files written:	29726
Chunks processed:	16 Files written:	32203
Chunks processed:	17 Files written:	34599
Chunks processed:	18 Files written:	37177
Chunks processed:	19 Files written:	40021
Chunks processed:	20 Files written:	43468
Chunks processed:	21 Files written:	46783


KeyboardInterrupt: 

In [28]:
thing = "hello, \' \n World."
thing2 = re.sub(r'[^A-Za-z0-9]+[\r\n]*', ' ', thing)
print thing
print thing2

hello, ' 
 World.
hello World 


In [ ]:
df = pd.read_sql('SELECT id, parent_id FROM May2015\
                WHERE LENGTH(body) > 10 AND LENGTH(body) < 150', db)